In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_page(url):
    response = requests.get(url)
    
    page_data = {
        'Product Name': [],
        'Regular Price': [],
        'Discounted Price': []
    }

    if response.status_code == 200:
        print(f"Berhasil mengambil halaman web: {url}")
        html_content = response.text

        soup = BeautifulSoup(html_content, 'html.parser')

        products = soup.find_all('div', class_='box-text box-text-products text-center grid-style-2')

        if not products:
            print("Tidak ada produk yang ditemukan di halaman ini!")
        else:
            print(f"{len(products)} produk ditemukan di halaman ini!")

        for product in products:
            try:
                product_name = product.find('p', class_='name product-title woocommerce-loop-product__title').text.strip()
            except:
                product_name = '-'
            
            sale_price = '-'
            regular_price = '-'

            del_tag = product.find('del')
            if del_tag:
                regular_price = del_tag.text.strip().replace('Rp', '').replace(',', '').strip()
                regular_price = "Rp {:,}".format(int(regular_price))  
            else:
                regular_price = '-'

            ins_tag = product.find('ins')
            if ins_tag:
                sale_price = ins_tag.text.strip().replace('Rp', '').replace(',', '').strip()
                sale_price = "Rp {:,}".format(int(sale_price))  # Format harga sale dengan koma
            else:
                sale_price = product.find('span', class_='woocommerce-Price-amount amount').text.strip().replace('Rp', '').replace(',', '').strip()
                sale_price = "Rp {:,}".format(int(sale_price))  # Format harga sale tanpa koma

            # Menambahkan data ke dalam list
            page_data['Product Name'].append(product_name)
            page_data['Regular Price'].append(regular_price)
            page_data['Discounted Price'].append(sale_price)

    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
    
    return page_data

base_url = "https://polytron.co.id/kategori-produk/tv/smart-tv/"

all_data = {
    'Product Name': [],
    'Regular Price': [],
    'Discounted Price': []
}

for page_num in range(1, 4):
    if page_num == 1:
        url = base_url 
    else:
        url = f"{base_url}page/{page_num}/"
    
    page_data = scrape_page(url)
    all_data['Product Name'].extend(page_data['Product Name'])
    all_data['Regular Price'].extend(page_data['Regular Price'])
    all_data['Discounted Price'].extend(page_data['Discounted Price'])
    

df = pd.DataFrame(all_data)

print(df.to_markdown(index=False))

Berhasil mengambil halaman web: https://polytron.co.id/kategori-produk/tv/smart-tv/
12 produk ditemukan di halaman ini!
Berhasil mengambil halaman web: https://polytron.co.id/kategori-produk/tv/smart-tv/page/2/
12 produk ditemukan di halaman ini!
Berhasil mengambil halaman web: https://polytron.co.id/kategori-produk/tv/smart-tv/page/3/
2 produk ditemukan di halaman ini!
| Product Name                                                           | Regular Price   | Discounted Price   |
|:-----------------------------------------------------------------------|:----------------|:-------------------|
| POLYTRON EQLED TV 100 inch 4K PLD 100QV5925                            | Rp 38,999,000   | Rp 33,900,000      |
| POLYTRON 4K UHD Smart Google TV 75 Inch PLD 75UG5959                   | Rp 14,999,000   | Rp 13,399,000      |
| POLYTRON 4K UHD Smart Google TV 65 Inch PLD 65UG5959                   | Rp 8,499,000    | Rp 7,599,000       |
| POLYTRON 4K UHD Smart Google TV 55 Inch PLD 55UG5959   